# Добавление остатков по магазинам

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re
from tqdm.notebook import trange, tqdm
from tqdm.gui import tqdm as tqdm_gui

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)

In [4]:
df = pd.read_csv('csv\hi_tech_2024_6_9.csv')
df

,soup_name,id,date,availability,title,price,old_price
0,https://hi-tech.md/bytovaya-tehnika/pylesosy-i-aksessuary/filtry-i-meshki-dlya-pyli/filtr-lg-vpm-sgm/,Т-000019420,2024-06-09,В наличии,Фильтр LG VPM-SGM,17.0,34.0
1,https://hi-tech.md/bytovaya-tehnika/tehnika-karcher/meshki-i-filtry-karcher/filtr-paket-karcher-6.904-084.0-flis-1sht-v-up-10/,Т-000051540,2024-06-09,В наличии,фильтр -пакет Karcher 6.904-084.0 флис 1шт (в уп 10),25.0,NaN
2,https://hi-tech.md/bytovaya-tehnika/pylesosy-i-aksessuary/aksessuary-k-pylesosam/meshok-dlya-robota-pylesosa-lydsto-r1/,Т-000083200,2024-06-09,В наличии,Мешок для робота пылесоса Lydsto R1,30.0,59.0
3,https://hi-tech.md/bytovaya-tehnika/aksessuary-dlya-melkoy-bytovoy-tehniki/aksessuary-dlya-zubnyh-schetok/nasadka-d-zub-schetki-panasonic-ew0940w830-2sht/,Т-000056675,2024-06-09,В наличии,насадка д/Зубная щетка Panasonic EW0940W830 2шт,34.0,68.0
4,https://hi-tech.md/bytovaya-tehnika/pylesosy-i-aksessuary/filtry-i-meshki-dlya-pyli/filtr-paket-karcher-6.904-333.0-1sht-v-up-10/,Т-000038497,2024-06-09,В наличии,фильтр -пакет Karcher 6.904-333.0 1шт (в уп 10),34.0,NaN
...,...,...,...,...,...,...,...
2549,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/kondicionery-kassetnye/kassetnyy-kondicioner-ballu-blcic-60hn8-eu23y/,Т-000091767,2024-06-09,В наличии,Кассетный кондиционер Ballu BLCI_C/in-60HN8/EU_23Y + BLCI_O/out-60HN8/EU + BLCI-PS/EU,40899.0,NaN
2550,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/promyshlennye-kondicionery/kanalnyy-kondicioner-electrolux-eacd-48h-up3-dc-n8-gree/,Т-000091331,2024-06-09,В наличии,Канальный кондиционер Electrolux EACD-48H/UP4-DC/N8 (GREE),41589.0,NaN
2551,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/promyshlennye-kondicionery/napolno-potolochnyy-kondicioner-electrolux-eacu-48h-up3-dc-n8-gree/,Т-000091326,2024-06-09,В наличии,Напольно-потолочный кондиционер Electrolux EACU-48H/UP4-DC/N8 (GREE),41699.0,NaN
2552,https://hi-tech.md/bytovaya-tehnika/klimaticheskaya-tehnika/promyshlennye-kondicionery/kanalnyy-kondicioner-ballu-blcid-60hn8-eu23y/,Т-000091778,2024-06-09,В наличии,Канальный кондиционер Ballu BLCI_D/in-60HN8/EU_23Y + BLCI_O/out-60HN8/EU,41899.0,NaN


In [5]:
df= df[df['id']=='Т-000071413']

In [6]:
df

,soup_name,id,date,availability,title,price,old_price
2447,https://hi-tech.md/bytovaya-tehnika/krupnaya-tehnika-dlya-kuhni/holodilniki/vstraivaemye-holodilniki/holodilnik-samsung-brb307054ww-ua/,Т-000071413,2024-06-09,В наличии,холодильник Samsung BRB307054WW/UA,13199.0,16499.0


In [7]:
def count_sity(x):
    reqx = requests.get(x)
    soup = bs(reqx.text, 'html.parser')    
    element = soup.findAll('div', class_='ty-warehouses-store__availability')
    data = []
    if len(element) == 14:
        data.append(element[0].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))    
        data.append(element[1].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[2].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[3].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[4].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0')) 
        data.append(element[5].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[6].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[7].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[8].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[9].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[10].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[11].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[12].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(element[13].text.strip().replace('В наличии', '1').replace('Нет в наличии','0')
        .replace('Под заказ (1-3 дня)','0').replace('nan','0'))
        data.append(0)        
        return data
    else:
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(0)
        data.append(1)
        return data


In [8]:
tqdm.pandas()
df['count_sity']= df['soup_name'].progress_apply(count_sity)

  0%|          | 0/1 [00:00<?, ?it/s]


(df['Тир_Юности_14А'],
 df['Ц_Тир_25_Октября_69'],
 df['Ц_Бенд_Спортивная_5'],
 df['Бенд_Советская_5'],
 df['Бенд_Советская_8'],
 df['Слоб_Ленина_97А'],
 df['Днестр_Строителей_18'],
 df['Первом_Ленина_88'],
 df['Григ_КМаркса_181А'],
 df['Григ_КМаркса_172'],
 df['Дуб_Горького_29'],
 df['Рыбн_Вальченко_43'],
 df['Рыбн_Кирова_135'],
 df['Кам_Кирова_240']) = df['soup_name'].apply(count_sity)

In [9]:
df['Тир_Юности_14А'] = df['count_sity'].str[0].astype('int')
df['Ц_Тир_25_Октября_69'] = df['count_sity'].str[1].astype('int')
df['Тирасполь'] = df['Тир_Юности_14А'] + df['Ц_Тир_25_Октября_69']
df['Бенд_Спортивная_5'] = df['count_sity'].str[2].astype('int')
df['Бенд_Советская_5'] = df['count_sity'].str[3].astype('int')
df['Бенд_Советская_8'] = df['count_sity'].str[4].astype('int')
df['Бендеры'] = df['Бенд_Спортивная_5'] + df['Бенд_Советская_5'] + df['Бенд_Советская_8']
df['Слободзея'] = df['count_sity'].str[5].astype('int')
df['Днестровск'] = df['count_sity'].str[6].astype('int')
df['Первомайск'] = df['count_sity'].str[7].astype('int')
df['Григ_КМаркса_181А'] = df['count_sity'].str[8].astype('int')
df['Григ_КМаркса_172'] = df['count_sity'].str[9].astype('int')
df['Дубоссары'] = df['count_sity'].str[10].astype('int')
df['Рыбн_Вальченко_43'] = df['count_sity'].str[11].astype('int')
df['Рыбн_Кирова_135'] = df['count_sity'].str[12].astype('int')
df['Рыбница'] = df['Рыбн_Вальченко_43'] + df['Рыбн_Кирова_135']
df['Кам_Кирова_240'] = df['count_sity'].str[13].astype('int')
df['error'] = df['count_sity'].str[14].astype('int')

In [10]:
df.head(15)

,soup_name,id,date,availability,title,price,old_price,count_sity,Тир_Юности_14А,Ц_Тир_25_Октября_69,Тирасполь,Бенд_Спортивная_5,Бенд_Советская_5,Бенд_Советская_8,Бендеры,Слободзея,Днестровск,Первомайск,Григ_КМаркса_181А,Григ_КМаркса_172,Дубоссары,Рыбн_Вальченко_43,Рыбн_Кирова_135,Рыбница,Кам_Кирова_240,error
2447,https://hi-tech.md/bytovaya-tehnika/krupnaya-tehnika-dlya-kuhni/holodilniki/vstraivaemye-holodilniki/holodilnik-samsung-brb307054ww-ua/,Т-000071413,2024-06-09,В наличии,холодильник Samsung BRB307054WW/UA,13199.0,16499.0,"[1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0]",1,1,2,0,1,0,1,0,0,0,1,0,1,0,1,1,0,0


In [11]:
df['Ц_Бенд_Спортивная_5'].unique()

KeyError: 'Ц_Бенд_Спортивная_5'

In [ ]:
today = pd.to_datetime('today').normalize()
today = today.to_pydatetime()
df.to_csv('csv/hi_tech_test_'+str(today.day)+'_'+str(today.month)+'_'+str(today.year)+'.csv', index=False)  